# Enhanced Vegetation Index Data Preprocessing

## Author: Yue 'Luna' Huang

### Nov 12, 2017

### Packages

* `pyhdf` can be installed from [here](http://fhs.github.io/python-hdf4/install.html).

```bash
sudo apt-get install libhdf4-dev
sudo pip3 install python-hdf4
```

* `GDAL` can be installed via `pip3 install`, despite some dependency issues. Codes are as below.

```bash
sudo apt-get install libgdal-dev
sudo pip3 install --global-option=build_ext --global-option="-I/usr/include/gdal" GDAL==2.2.1
```

* Run `jupyter notebook` on the remote server and connect it to localhost.

```bash
# login
ssh -L localhost:8888:localhost:8888 kramer
# do not launch browser in terminal
jupyter notebook --no-browser
```

Then follow the prompt: Copy/paste this URL into your browser when you connect for the first time, to login with a token.

### Data

* We obtained data from [DATA SOURCE](https://search.earthdata.nasa.gov/search/granules?p=C194001220-LPDAAC_ECS&tl=1494721209!4!!&q=MYD13C1&ok=MYD13C1). Official documentation can be found at [DOCS](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table/myd13c1), and [MOD13C2 DOCS](https://lpdaac.usgs.gov/dataset_discovery/modis/modis_products_table/mod13c2_v006) is also informative.
* We extract `CMG 0.05 Deg 16 days EVI`, whose fill value is -3000, valid range is -2000 to 10000, and scaling factor is 0.0001.
* We extract `CMG 0.05 Deg Monthly EVI std dev`, which is standard deviation computed from input EVI pixels, whose fill value is -3000, valid range is 0 to 10000, and scaling factor is 0.0001.
* We also examine `CMG 0.05 Deg 16 days pixel reliability` (-1--4) and just drop missing values.
    * -1: No Data
    * 0: Good Data (Use with confidence)
    * 1: Marginal data (Useful, but look at other QA information)
    * 2: Snow/Ice
    * 3: Cloudy
    * 4: Estimated from MODIS historic time series

### Downloads

After obtaining an `evi_url_list.txt` file containing all the links to downloading the `.hdf` files, I ran the following bash script to automatically download all the data.

```bash
# change directory
cd /disk/evi
# silently run the commands in the background
nohup wget --content-disposition --http-user=my_username --http-password=my_password --keep-session-cookies -i evi_url_list.txt &
# a nohup.out file will be generated to document downloading progress
tail -f nohup.out
# remove when finishing downloading
rm nohup.out
```

In [1]:
# load libraries
import os, sys
import numpy as np
import pprint
from datetime import datetime
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt

from osgeo import gdal, osr
from pyhdf.SD import SD, SDC

In [2]:
assert sys.version_info[0] >= 3, "Python 3 or a more recent version is required."

## Classes and Functions

In [3]:
class Raster:
    
    import numpy as np
    
    def __init__(self,
                 latUpper=56 - 0.1/2, longLower=72 + 0.1/2,
                 latStep=0.1, longStep=0.1,
                 latN=400, longN=650):
        """ Record latitude and longitude information on the uniform grid.
        Default to our bounding box in China.
        
        :param
            latUpper, longLower:
                float, the upper left corner of the bounding box
            latStep, longStep:
                float, the spatial resolution in degrees, both should be positive
            latN, longN:
                int, total number of observations in each dimension
        """
        
        # initialized values
        self.latUpper = latUpper
        self.longLower = longLower
        self.latStep = latStep
        self.longStep = longStep
        self.latN = latN
        self.longN = longN
        self.latLower = latUpper - latStep * (latN - 1)
        self.longUpper = longLower + longStep * (longN - 1)
        # compute coordinates
        self.lat = np.linspace(self.latLower, self.latUpper, self.latN)
        self.long = np.linspace(self.longLower, self.longUpper, self.longN)
        # initialize empty array
        self.values = np.empty((self.longN, self.latN))
        self.values.fill(np.nan)
        
    def fill(self, array):
        """ Fill in the raster with the array.
        
        :param
            array:
                2 dimensional ndarray, the first element of the array should be
                the lower left corner of the map (smallest long and lat)
                the array is long (dimension 0) by lat (dimension 1)
        """
        
        # check dimension
        assert array.shape == (self.longN, self.latN), "Dimension Mismatch."
        # copy array
        self.values = array.copy()
    
    def find_neighbor(self, source_raster, method):
        """ Impute the closest observation contained in the source_raster onto this grid.
        This is recommended for fine grids.
        
        :param
            source_raster:
                a Raster object, values will be taken from the source raster,
                output array will be written into self.values
            method:
                a list of strings, should contain either 'upper' or 'lower' and either 'left' or 'right'
                nearst neighbor (from source_raster) will be to the (say) 'upper_left' of coordinates of self
        """
        
        # extract values from source array via direct indexing
        for selfLongIndex in range(self.longN):
            # find source index
            if 'left' in method:
                sourceLongIndex = int(np.floor(
                    (self.long[selfLongIndex] - source_raster.longLower) / source_raster.longStep))
            elif 'right' in method:
                sourceLongIndex = int(np.ceil(
                    (self.long[selfLongIndex] - source_raster.longLower) / source_raster.longStep))
            else:
                raise Exception("Methods Unspecified!")
                
            for selfLatIndex in range(self.latN):  
                # find source index
                if 'lower' in method:
                    sourceLatIndex = int(np.floor(
                        (self.lat[selfLatIndex] - source_raster.latLower) / source_raster.latStep))
                elif 'upper' in method:
                    sourceLatIndex = int(np.ceil(
                        (self.lat[selfLatIndex] - source_raster.latLower) / source_raster.latStep))
                else:
                    raise Exception("Methods Unspecified!")
                # impute value
                self.values[selfLongIndex, selfLatIndex] = source_raster.values[sourceLongIndex, sourceLatIndex]

In [4]:
def array2raster(array, geotiffFile=None, path=".", varName=None, date=None, date_format="%Y%j",
                 rasterOrigin=(72, 56), pixelLatWidth=0.1, pixelLonWidth=0.1):
    """ This function converts array to a raster and saves it to a GeoTIFF file.
    
    :params
        array:
            2-dimensional float ndarray, lat-long (x should be lat), the lower left (south-western) corner
            should be coded as the first element in the array
        geotiffFile:
            string, path and file name for the GeoTIFF file, this option overrides path, varName and date
        path:
            string, instead of passing in the geotiffFile argument, let the function automatically generate
            file names within this path
        varName:
            string, instead of passing in the geotiffFile argument, let the function automatically generate
            file names with this variable name
            should be in the format of "XXX_XXX", e.g. "MODIS_EVI"
        date:
            string, instead of passing in the geotiffFile argument, let the function automatically generate
            file names with this date, date could be like "2005-01-01" or "2005001" (day of year)
        date_format:
            string, specify format for date if date is provided, defaults to day of year (e.g. "2005001")
            see: https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime
        rasterOrigin:
            float tuple, the lon-lat coordinate of the origin of the raster
            (the largest lat / smallest lon coordinates, i.e. upper left corner)
            defaults to the origin of the bounding box in this project
        pixelLatWidth, pixelLonWidth:
            float, how many degrees latitude or longitude each element in the array reprensents
            this is spatial resolution for the array
            defaults to 0.1 degree by 0.1 degree
    """
    
    from datetime import datetime
    import gdal, osr
    import os
    
    # assign file name and path
    if geotiffFile is None:
        if varName is None or date is None:
            raise Exception("Please supply either the geotiffFile argument " +
                            "or both arguments of varName and date.")
        else:
            # automatically convert dates
            file_path = os.path.abspath(os.path.join(
                path, datetime.strptime(date, date_format).strftime("%Y-%m-%d") + "_" + varName + ".tif"))
    else:
        file_path = os.path.abspath(os.path.join(
                path, geotiffFile))
        
    # extract bounding box
    try:
        rows, cols = array.shape
    except:
        raise Exception("Array needs to be two dimensional.")
    originLon, originLat = rasterOrigin
    
    # write files
    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(file_path, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originLon, pixelLonWidth, 0,
                               originLat, 0, - pixelLatWidth))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    
    # georeference
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromProj4('+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0')
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    
    # close files
    outRaster = None

## Data Cleaning

All data files on kramer are under `/disk/yogasaur/`. The subdirectory `/home/maxjiang/yogasaur/codes/` is synced with the git repository. EVI data are in `/disk/yogasaur/evi/`.

In [5]:
# define input file path
file_root = os.path.abspath("/disk/yogasaur/evi/")
# define dataset name
dataset_name = "MYD13C1"
# define variables to extract
var_name_list = [
    'CMG 0.05 Deg 16 days EVI',
    'CMG 0.05 Deg 16 days EVI std dev']
# define output file path
output_root = os.path.abspath("/disk/yogasaur/merge/evi/")
# define output variable names
output_var_name_list = [
    'MODIS_EVI', 'MODIS_EVISD']
# define fill value
fill_value = -3000
# define scaling factor
scaling_factor = 0.0001

In [6]:
# construct file list
files = [f for f in os.listdir(file_root) if f.startswith(dataset_name)]

In [7]:
# examine a few files
if False:
    file = files[3]
    # load hdf file
    hdf = SD(os.path.join(file_root, file), SDC.READ)
    # pretty print
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(hdf.datasets())
    for var_name in var_name_list + ['CMG 0.05 Deg 16 days pixel reliability']:
        # extract variable
        var = hdf.select(var_name).get()
        # flip longitude and latitude, flip latitude
        var = np.flip(var.T, axis=1)
        # visualize
        plt.imshow(var.T, origin='lower')
        plt.colorbar()
        plt.show()

In [8]:
# test first few cases
if True:
    files = files[0:2]

In [9]:
# instantiate a raw raster
raw_raster = Raster(
    latUpper=90 - 0.05/2, longLower=-180 + 0.05/2,
    latStep=0.05, longStep=0.05,
    latN=3600, longN=7200)

# initiate empty output raster
output_raster = Raster()

for file in tqdm_notebook(files, desc="Processing"):
    # parse file name, extract date
    _, date, _, _, _ = file.split(sep=".")
    date = date[1:]
    # load hdf files
    hdf = SD(os.path.join(file_root, file), SDC.READ)
    for i, var_name in enumerate(var_name_list):
        # extract variable
        var = hdf.select(var_name).get()
        # flip longitude and latitude, flip latitude, cast to float
        var = np.flip(var.T, axis=1).astype(float)
        # scale and drop fill values
        var[var == fill_value] = np.nan
        var = var * scaling_factor
        # fill the raw raster with the source array
        raw_raster.fill(var)
        # using different methods, construct four maps
        for method, varSuffix in [(['upper', 'left'], 'UL'),
                                  (['upper', 'right'], 'UR'),
                                  (['lower', 'left'], 'LL'),
                                  (['lower', 'right'], 'LR')]:
            # impute values (closest neighbors)
            output_raster.find_neighbor(source_raster=raw_raster, method=method)
            # save output
            array2raster(array=output_raster.values,
                         path=output_root, varName=output_var_name_list[i] + "-" + varSuffix, date=date)

HBox(children=(IntProgress(value=0, description='Processing', max=2), HTML(value='')))